# Noodling with ideas from On Numbers and Games

The notation introduced in On Numbers, brief demo for my future usage:

In [1]:
(require '[clojupyter.misc.display :as display])
(display/markdown "\\begin{equation}
\\{ \\emptyset | \\emptyset \\}
\\end{equation}")

\begin{equation}
\{ \emptyset | \emptyset \}
\end{equation}

Need to come up with a manageable way not to have to do all escape characters all the time.

\begin{equation}
\{\emptyset|\emptyset\}
\end{equation}

In [2]:
(defn simple-conway-notation [n1 n2] (str "\\begin{equation} \\{" n1 "|" n2 "\\}\\end{equation}"))

#'user/simple-conway-notation

In [3]:
(display/markdown (simple-conway-notation "\\emptyset" "\\emptyset"))

\begin{equation} \{\emptyset|\emptyset\}\end{equation}

In [4]:
(display/markdown (simple-conway-notation 0 1))

\begin{equation} \{0|1\}\end{equation}

## Is it a number?
Conway says that if an expression contains only numbers on the left and the right and if there is no number on the left equal to or greater than any number on the left, then the resulting expression is considered equal to a number.  Let's assume it's ok for now to represent the notation from the book as two Clojure vectors.

In [5]:
(defn is-number [left right]
    (if (= 0 (count right))
           true
           (let [least-r (reduce min right)
                 greatest-l (reduce max left)]
               (> least-r greatest-l))))

#'user/is-number

In [6]:
(is-number [1 2 3] [4 5 6])

true

In [7]:
(is-number [1 2 3] [])

true

In [8]:
(is-number [2] [1])

false

A simple case of two nonempty sets that fit the rule, then an empty set for the right that can only pass, then an example of something that is not a number.  A minor improvement must be made to the notation code to display some of these scenarios:

In [9]:
(defn simple-conway-notation [n1 n2]
    (str "\\begin{equation} \\{"
         (clojure.string/join "," n1)
         "|"
         (clojure.string/join "," n2)
         "\\}\\end{equation}"))

#'user/simple-conway-notation

In [10]:
(defn ask-is-num [left right]
    (str (simple-conway-notation left right) " "
         (if (is-number left right) "is" "is not")
         " a number"))

#'user/ask-is-num

In [11]:
(display/markdown (str
                   (ask-is-num [1 2 3] [4 5 6])
                   (ask-is-num [1 2 3] [])
                   (ask-is-num [2] [1])))

\begin{equation} \{1,2,3|4,5,6\}\end{equation} is a number\begin{equation} \{1,2,3|\}\end{equation} is a number\begin{equation} \{2|1\}\end{equation} is not a number

## Constructing some numbers in the new notation.
Taking some numbers from the constructions for basic integers in the book and fully expanding them as lists of lists proves somewhat interesting.

In [12]:
(def empty-set [])
(def zero [empty-set :bar empty-set])
(def one [[zero] :bar empty-set])
(def two [[one] :bar empty-set])
(def neg-one [empty-set :bar [zero]])

#'user/neg-one

In [13]:
two

[[[[[[] :bar []]] :bar []]] :bar []]

In [14]:
zero

[[] :bar []]

In [15]:
neg-one

[[] :bar [[[] :bar []]]]

In [16]:
(defn incr [exp] [exp :bar []])

#'user/incr

In [17]:
(incr two)

[[[[[[[] :bar []]] :bar []]] :bar []] :bar []]

In [18]:
(incr (incr two))

[[[[[[[[] :bar []]] :bar []]] :bar []] :bar []] :bar []]

In [19]:
(defn decr [exp] [[] :bar exp])

#'user/decr

In [20]:
(decr neg-one)

[[] :bar [[] :bar [[[] :bar []]]]]

In [21]:
(decr (decr neg-one))

[[] :bar [[] :bar [[] :bar [[[] :bar []]]]]]

The integers are symmetrical when you look at their negatives.  This kind of makes sense.  I thought it would be interesting to see the fully expanded set notation itself:

In [22]:
(defn get-left [expr] (into [] (take-while #(not (= % :bar)) expr)))
(defn get-right [expr] (drop (+ 1 (count (get-left expr))) expr))

(defn simple-conway-notation [expr]
    (cond
        (= 0 (count expr))
        "\\emptyset"
        (some #(= % :bar) expr)
        (let [pre-bar (get-left expr)
              post-bar (get-right expr)]
            (str
             "\\{"
             (clojure.string/join "," (map simple-conway-notation pre-bar))
             "|"
             (clojure.string/join "," (map simple-conway-notation post-bar))
             "\\}"))
        :else
        (clojure.string/join "," (map simple-conway-notation expr))))

#'user/simple-conway-notation

In [23]:
(display/markdown (str "\\begin{equation} -2 =" (simple-conway-notation (decr neg-one)) "\\end{equation}"))

\begin{equation} -2 =\{\emptyset|\{\emptyset|\{\emptyset|\emptyset\}\}\}\end{equation}

May as well make things easier

In [24]:
(defn print-eq [note expr] 
    (display/markdown (str "\\begin{equation}"
                           note " "
                           (simple-conway-notation expr)
                           "\\end{equation}")))

#'user/print-eq

In [25]:
(print-eq "2 =" two)

\begin{equation}2 = \{\{\{\emptyset|\emptyset\}|\emptyset\}|\emptyset\}\end{equation}

## The $\ge$ implementation
ONAG specifies the gte relation in terms of itself; it is recursive, but there is a base case.

For inputs $x = \{x^L|x^R\}$ and $y = \{y^L|y^R\}$ Conway has constructed a definition of this relation that depends on the existence of the empty set. At some point in the process, he is counting on $x^R$ and $y^L$ resolving to the $\emptyset$ and thereby trivially satisfying the "no element of .... satisfies (whatever relation)" because .... has no elements.

Conway's definition says for the relation $x \ge y$ to hold, there must be no $x^R \leq y$ and no $y^L \ge x$

In [26]:
(print-eq "1 = " one)

\begin{equation}1 =  \{\{\emptyset|\emptyset\}|\emptyset\}\end{equation}

In [27]:
(print-eq "0 = " zero)

\begin{equation}0 =  \{\emptyset|\emptyset\}\end{equation}

In order to establish that $1 \ge 0$ we must know $1^R=\emptyset$ and $0^L=\emptyset$

Since there are no elements in $0^L$, that part trivially passes.

$1^R$ is also $\emptyset$ so that also trivially passes, so now we know $1 \ge 0$.  In this way I am starting to understand the notation a little better -- if $\{set1|set2\}$ is a number, then that number is the value _exactly between_ those two sets.  This does not produce a unique value for every combination of sets, but the equality relation actually comes later.  Right now we do not even know that!

Now for a stab at a gte base case implentation, assuming both expr1 and expr2 are numbers:

In [28]:
(defn satisfies-gte-base-case [exp1 exp2] ; read as: testing whether exp1 gte exp2 according to the base case
    (let [e1-right (into [] (get-right exp1))
          e2-left (into [] (get-left exp2))]
        (and (= 1 (count e1-right))
             (= 1 (count e2-left))
             (= 0 (count (e1-right 0)))
             (= 0 (count (e2-left 0))))))

#'user/satisfies-gte-base-case

In [29]:
(satisfies-gte-base-case one zero)

true

In [30]:
(satisfies-gte-base-case zero neg-one)

true

In [31]:
(satisfies-gte-base-case zero one)

false

In [32]:
(satisfies-gte-base-case neg-one zero)

false

Initially I thought these two trues were the only uses of the base case, but there are more.

In [33]:
(satisfies-gte-base-case two zero)

true

In [34]:
(satisfies-gte-base-case zero (decr neg-one))

true

Since it's greater than _or equal_, we should expect some equals cases to pass:

In [35]:
(satisfies-gte-base-case zero zero)

true

but really only that one above....

In [36]:
(satisfies-gte-base-case one one)

false

There are plenty of cases that require the recursive step though, as these will be true once that is in place:

In [37]:
(satisfies-gte-base-case two one)

false

In [38]:
(def half [zero :bar one])

#'user/half

In [39]:
(satisfies-gte-base-case half zero)

false

The recursive implementation will take the same arguments, of course.  Again for $x \ge y$ to hold, there must be no $x^R \leq y$ and no $y^L \ge x$.  This implementation seems to be partially correct -- see the examples below

In [40]:
(defn satisfies-gte-simple [x-expression y-expression] ; again reading as is-exp1-gte-exp2?
    (or (satisfies-gte-base-case x-expression y-expression)
        (let [all-x-r (get-right x-expression)
              all-y-l (get-left y-expression)
              violating-x-r (some #(satisfies-gte-simple y-expression %) all-x-r)
              violating-y-l (some #(satisfies-gte-simple % x-expression) all-y-l)]
            (and (not violating-x-r) (not violating-y-l)))))

#'user/satisfies-gte-simple

In [41]:
(satisfies-gte-simple two one)

true

In [42]:
(satisfies-gte-simple half zero) ; this does not result in the correct output

false

In [43]:
(satisfies-gte-simple one neg-one)

true

In [73]:
(defn solve-gt-scenario-template [exp1 exp2]
    {
     :problem-stack '([exp1 exp2])
     :solved {}
     :subproblems {}
    })

#'user/solve-gt-scenario-template

In [81]:
(defn get-gt-subproblems [x-expression y-expression]
    (let [all-x-r (get-right x-expression)
          all-y-l (get-left y-expression)]
        (concat
         (map #(vector y-expression %) all-x-r)
         (map #(vector % x-expression) all-y-l))))

#'user/get-gt-subproblems

In [82]:
(get-gt-subproblems two neg-one)

([[[] :bar [[[] :bar []]]] []] [[] [[[[[[] :bar []]] :bar []]] :bar []]])

In [88]:
(map #(satisfies-gte-base-case (% 0) (% 1)) (get-gt-subproblems one zero))

(false false)